# Import Necessary Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests as rq # for requests

from bs4 import BeautifulSoup # for scraping


# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    certifi-2018.11.29         |        py36_1000         145 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

## Select URL

In [4]:
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(website_url, auth=('user', 'pass'))

In [21]:
reqText = req.text
soup = BeautifulSoup(reqText, 'html.parser')

## Create DataFrame

In [376]:
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
#print(df[0].to_json(orient='records'))

In [377]:
df = df[0]

In [378]:
df.columns = ['Postal_Code','Borough','Neighbourhood']

In [379]:
df = df.drop(0)
df.head()

,Postal_Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [380]:
dfClean = df.loc[df['Borough'] != 'Not assigned']
dfClean.head()

,Postal_Code,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [381]:
dfClean = dfClean.reset_index()

In [382]:
dfClean.head()

,index,Postal_Code,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M5A,Downtown Toronto,Regent Park
4,7,M6A,North York,Lawrence Heights


In [383]:
dfClean = dfClean.drop(columns='index')
dfClean.head()

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [384]:
dfClean.columns

Index(['Postal_Code', 'Borough', 'Neighbourhood'], dtype='object')

In [385]:
dfClean.head()

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [386]:
idx = dfClean.loc[dfClean['Neighbourhood'] == 'Not assigned']

In [387]:
dfClean.iloc[idx.index[0]].Neighbourhood = dfClean.iloc[idx.index[0]].Borough

In [388]:
dfClean.head()

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [389]:
dfClean.columns

Index(['Postal_Code', 'Borough', 'Neighbourhood'], dtype='object')

### Find Postcodes

In [390]:
dfClean.sort_values(['Postal_Code'])
dfClean.head()

,Postal_Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [394]:
dfClean = dfClean.groupby(['Postal_Code','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

In [399]:
dfClean = pd.DataFrame(dfClean)

In [402]:
dfClean = dfClean.reset_index()
dfClean.head()

,Postal_Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [403]:
dfClean.values

array([['M1B', 'Scarborough', 'Rouge, Malvern'],
       ['M1C', 'Scarborough', 'Highland Creek, Rouge Hill, Port Union'],
       ['M1E', 'Scarborough', 'Guildwood, Morningside, West Hill'],
       ['M1G', 'Scarborough', 'Woburn'],
       ['M1H', 'Scarborough', 'Cedarbrae'],
       ['M1J', 'Scarborough', 'Scarborough Village'],
       ['M1K', 'Scarborough',
        'East Birchmount Park, Ionview, Kennedy Park'],
       ['M1L', 'Scarborough', 'Clairlea, Golden Mile, Oakridge'],
       ['M1M', 'Scarborough',
        'Cliffcrest, Cliffside, Scarborough Village West'],
       ['M1N', 'Scarborough', 'Birch Cliff, Cliffside West'],
       ['M1P', 'Scarborough',
        'Dorset Park, Scarborough Town Centre, Wexford Heights'],
       ['M1R', 'Scarborough', 'Maryvale, Wexford'],
       ['M1S', 'Scarborough', 'Agincourt'],
       ['M1T', 'Scarborough', "Clarks Corners, Sullivan, Tam O'Shanter"],
       ['M1V', 'Scarborough',
        "Agincourt North, L'Amoreaux East, Milliken, Steeles East"],
  

### Get Lat/Long

In [404]:
dfCoord = pd.read_csv('http://cocl.us/Geospatial_data') 
dfCoord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [370]:
dfClean.columns

Index(['Neighbourhood'], dtype='object')

In [349]:
dfCoord.columns = ['Postal_Code','Latitude','Longitude']
dfCoord.head()

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [366]:
dfMerge = dfClean.merge(dfCoord, left_on='Postal_Code', right_on='Postal_Code', how='left')
dfMerge.head()

,Postal_Code,Neighbourhood,Latitude,Longitude
0,M1B,"Rouge, Malvern",43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Woburn,43.770992,-79.216917
4,M1H,Cedarbrae,43.773136,-79.239476


In [360]:
dfClean.head()

,,Neighbourhood
Postal_Code,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
